# Navie Bayes Practice

## 직접 구현
- 긍정(1), 부정(0)
- 새로운 메일이 왔는데 'happy weekend'가 포함되어 있을 경우, 이 메일은 긍정인가, 부정인가?

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('naivebayes_example.csv')
df

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [2]:
# 토큰화

from collections import defaultdict
word2id = defaultdict(lambda : len(word2id))

doc_ls = [doc.split() for doc in df['mail']]

for doc in doc_ls:
    for token in doc:
        word2id[token]
word2id

defaultdict(<function __main__.<lambda>()>,
            {'i': 0,
             'love': 1,
             'you': 2,
             'happy': 3,
             'weekend': 4,
             'bore': 5,
             'work': 6,
             'job': 7,
             'hate': 8,
             'together': 9})

In [3]:
# label 분류에 등장한 빈도수 계산

Bayes = np.zeros((len(word2id), int(df['label'].value_counts().count())*2))

for i, doc in enumerate(df['mail']):
    doc=doc.split()
    for token in doc:
        if (df['label'][i] == 1):
            Bayes[word2id[token], 0] += 1
        elif (df['label'][i] == 0):
            Bayes[word2id[token], 1] += 1

In [4]:
Bayes

array([[1., 1., 0., 0.],
       [2., 0., 0., 0.],
       [1., 1., 0., 0.],
       [2., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 2., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [5]:
# 조건부확률 계산
for i in range(len(Bayes)):
    Bayes[i][2]=(Bayes[i][0]+1)/(Bayes[:,0].sum()+len(word2id))
    Bayes[i][3]=(Bayes[i][1]+1)/(Bayes[:,1].sum()+len(word2id))

In [14]:
# happy, weekend의 긍정, 부정에 대한 기대값
import math
char1 = 'happy weekend'
char=char1.split()

sample = np.zeros((len(char),len(Bayes[0])))

for i, token in enumerate(char):
    for j in range(len(Bayes[0])):
        sample[i][j] = Bayes[word2id[token], j]
        
exp_p_1 = math.exp(math.log(sample[0][2]*sample[1][2]*0.5))
exp_p_2 = math.exp(math.log(sample[0][2]*sample[1][2]*0.5))+math.exp(math.log(sample[0][3]*sample[1][3]*0.5))
exp_p = exp_p_1/exp_p_2

exp_n_1 = math.exp(math.log(sample[0][3]*sample[1][3]*0.5))
exp_n_2 = math.exp(math.log(sample[0][2]*sample[1][2]*0.5))+math.exp(math.log(sample[0][3]*sample[1][3]*0.5))
exp_n = exp_n_1/exp_n_2

print('문자열 {} 일 때, 긍정일 확률 {}, 부정일 확률 {}'.format(char1, exp_p, exp_n))

문자열 happy weekend 일 때, 긍정일 확률 0.7499999999999999, 부정일 확률 0.2500000000000001


In [7]:
sample

array([[2.        , 0.        , 0.16666667, 0.05555556],
       [1.        , 1.        , 0.11111111, 0.11111111]])

In [8]:
Bayes

array([[1.        , 1.        , 0.11111111, 0.11111111],
       [2.        , 0.        , 0.16666667, 0.05555556],
       [1.        , 1.        , 0.11111111, 0.11111111],
       [2.        , 0.        , 0.16666667, 0.05555556],
       [1.        , 1.        , 0.11111111, 0.11111111],
       [0.        , 2.        , 0.05555556, 0.16666667],
       [0.        , 1.        , 0.05555556, 0.11111111],
       [0.        , 1.        , 0.05555556, 0.11111111],
       [0.        , 1.        , 0.05555556, 0.11111111],
       [1.        , 0.        , 0.11111111, 0.05555556]])

- 다음의 코드부터는 강의 자료에서 참고한 코드이다.

In [15]:
input_file = pd.read_csv("naivebayes_example.csv") # 읽어오기 
input_file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [16]:
training_set = np.array(input_file)
print(training_set)

[['i love you' 1]
 ['love happy weekend' 1]
 ['bore work job' 0]
 ['i hate you' 0]
 ['bore weekend' 0]
 ['happy together' 1]]


- 토큰 빈도수 및 문서별 토큰수 계산
   - 확률 계산을 위한 준비

In [17]:
# 토큰별로 문서 내 빈도수 카운팅
from collections import defaultdict
wordfreq = defaultdict(lambda : [0,0])  # lambda : 새로운 단어가 추가 될때 default 를 [0,0]으로 하겠다. [긍정,부정]
for doc, point in training_set:
    words = doc.split()
    for word in words:
        if point == 1:   
            wordfreq[word][0] +=1 # 긍정인 경우 0번째 인텍스에 1더해라 
        else:
            wordfreq[word][1] +=1 # 부정인 경우 1번째 인텍스에 1더해라
        
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [18]:
# 긍정/ 부정 빈도수 계산
긍정전체토큰수=[]
부정전체토큰수=[]
for key, (cnt1,cnt0) in wordfreq.items():
    긍정전체토큰수.append(int(cnt1))
    부정전체토큰수.append(int(cnt0))
print(긍정전체토큰수)
print(부정전체토큰수)
전체갯수_긍정 = sum(긍정전체토큰수)
전체갯수_부정 = sum(부정전체토큰수)
print(전체갯수_긍정,전체갯수_부정)

[1, 2, 1, 2, 1, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 2, 1, 1, 1, 0]
8 8


- Training : 토큰별 조건부 확률 계산

In [19]:
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [20]:
wordprobs = defaultdict(lambda:[0,0])
for key, (cnt1,cnt0) in wordfreq.items():
    wordprobs[key][0] = (cnt1 + 1)/(전체갯수_긍정 + len(wordfreq))
    wordprobs[key][1] = (cnt0 + 1)/(전체갯수_부정 + len(wordfreq))
wordprobs

defaultdict(<function __main__.<lambda>()>,
            {'i': [0.1111111111111111, 0.1111111111111111],
             'love': [0.16666666666666666, 0.05555555555555555],
             'you': [0.1111111111111111, 0.1111111111111111],
             'happy': [0.16666666666666666, 0.05555555555555555],
             'weekend': [0.1111111111111111, 0.1111111111111111],
             'bore': [0.05555555555555555, 0.16666666666666666],
             'work': [0.05555555555555555, 0.1111111111111111],
             'job': [0.05555555555555555, 0.1111111111111111],
             'hate': [0.05555555555555555, 0.1111111111111111],
             'together': [0.1111111111111111, 0.05555555555555555]})

- Classify : 신규텍스트가 주어졌을 때 확률 계산

In [21]:
import math
doc = "happy weekend" 
tokens = doc.split()
tokens

['happy', 'weekend']

In [22]:
# 초기값을 모두 0으로 처리 
log_prob1 = log_prob0 = 0.0

# 모든 단어에 대해 반복 
for word, (prob1,prob0) in wordprobs.items():
    if word in tokens:
        log_prob1 +=math.log(prob1)
        log_prob0 +=math.log(prob0)
log_prob1 += math.log(전체갯수_긍정/(전체갯수_긍정+전체갯수_부정))
log_prob0 += math.log(전체갯수_부정/(전체갯수_긍정+전체갯수_부정))

prob1 = math.exp(log_prob1) 
print('prob1',prob1)
prob0 = math.exp(log_prob0)   
print('prob0',prob0)
print('happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률')
print("긍정확률 : {}%".format(prob1/(prob1+prob0)*100))
print("부정확률 : {}%".format(prob0/(prob1+prob0)*100))


prob1 0.009259259259259257
prob0 0.0030864197530864183
happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률
긍정확률 : 75.00000000000001%
부정확률 : 24.999999999999996%


   # Using sklearn

In [9]:
X_train = list(df['mail'])
Y_train = list(df['label'])
print(X_train)
print(Y_train)

['i love you', 'love happy weekend', 'bore work job', 'i hate you', 'bore weekend', 'happy together']
[1, 1, 0, 0, 0, 1]


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

# CountVectorizer 선언
count_vect = CountVectorizer()

# fit and transform
X_train_counts = count_vect.fit_transform(X_train)

# MultinomialNB 선언 and fir
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [11]:
# 예측
print(clf.predict(count_vect.transform(['happy weekend'])))

# 확률
print(clf.predict_proba(count_vect.transform(['happy weekend'])))

[1]
[[0.25 0.75]]
